In [1]:
import numpy as np
import pandas as pd

In [9]:
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
import tensorflow as tf

In [6]:
import tensorflow_recommenders as tfrs

In [7]:
movies_df = pd.read_csv("/Users/prajualpillai/Desktop/prajual/Personal_git/Recommendation_system/collaborative_recommendation_system/movies.csv")
ratings_df = pd.read_csv("/Users/prajualpillai/Desktop/prajual/Personal_git/Recommendation_system/collaborative_recommendation_system/ratings.csv")

In [8]:
merged_df = movies_df.merge(ratings_df, on=["movieId"])
merged_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [55]:
ratings = np.array(merged_df["rating"].values.tolist())
ratings = tf.data.Dataset.from_tensor_slices(ratings)
ratings

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float64, name=None)>

In [56]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [57]:
# movies = movies_df.values.tolist()
import numpy as np
movies = tf.data.Dataset.from_tensor_slices(np.array(merged_df.values.tolist()))
movies

<TensorSliceDataset element_spec=TensorSpec(shape=(6,), dtype=tf.string, name=None)>

In [16]:
import tensorflow_datasets as tfds

/Users/prajualpillai/miniforge3/envs/health/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
from typing import Dict, Text
class Model(tfrs.Model):

  def __init__(self):
    super().__init__()

    # Set up user representation.
    self.user_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up movie representation.
    self.item_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.item_model(features["movie_id"])

    return self.task(user_embeddings, movie_embeddings)


In [59]:
model = Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [60]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

In [61]:
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [62]:
model.fit(train.batch(4096), epochs=5)

Epoch 1/5


TypeError: in user code:

    File "/Users/prajualpillai/miniforge3/envs/health/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/prajualpillai/miniforge3/envs/health/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/prajualpillai/miniforge3/envs/health/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/prajualpillai/miniforge3/envs/health/lib/python3.9/site-packages/tensorflow_recommenders/models/base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "/var/folders/05/mz9vyrj559z6gnm1ptm7x5z80000gq/T/ipykernel_19079/3961122226.py", line 23, in compute_loss
        user_embeddings = self.user_model(features["user_id"])

    TypeError: O

In [41]:
train, info = tfds.load('movielens/100k-ratings', split="train", with_info=True)
info

tfds.core.DatasetInfo(
    name='movielens',
    full_name='movielens/100k-ratings/0.1.1',
    description="""
    This dataset contains a set of movie ratings from the MovieLens website, a movie
    recommendation service. This dataset was collected and maintained by [GroupLens]
    (https://grouplens.org/), a research group at the University of Minnesota. There
    are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all
    datasets, the movies data and ratings data are joined on "movieId". The 25m
    dataset, latest-small dataset, and 20m dataset contain only movie data and
    rating data. The 1m dataset and 100k dataset contain demographic data in
    addition to movie and rating data.
    
    - "25m": This is the latest stable version of the MovieLens dataset. It is
    recommended for research purposes.
    - "latest-small": This is a small subset of the latest version of the MovieLens
    dataset. It is changed and updated over time by GroupLens.
    - "10

In [19]:
type(sample)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [36]:
sample1 = sample.map(lambda x: tf.strings.to_number(x["movie_id"]))

In [37]:
for x in sample.take(1).as_numpy_iterator():
    print(x)

{'bucketized_user_age': 45.0, 'movie_genres': array([7]), 'movie_id': b'357', 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'raw_user_age': 46.0, 'timestamp': 879024327, 'user_gender': True, 'user_id': b'138', 'user_occupation_label': 4, 'user_occupation_text': b'doctor', 'user_rating': 4.0, 'user_zip_code': b'53211'}


2022-09-19 16:07:07.424798: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [38]:
for x in movies.take(1).as_numpy_iterator():
    print(x)

AttributeError: 'list' object has no attribute 'take'

In [47]:
autotune = tf.data.experimental.AUTOTUNE

In [ ]:
custom_t = movies_df